In [ ]:
"""
Extract daily patch metrics from DattoRMM API, including device data transformation, patch status shaping,
compliance reporting, and CSV export for service delivery metrics.
"""


<h1> Daily MS Patch Metrics </h1>

# DataFrame Creation from API
## Prepare for DataFrame Creation
### Set Export Folder

In [ ]:
# Data Shaping
import pandas as pd
import numpy as np
import re
import datetime as dt

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# File Handling and Export
import json
import csv
import xlrd

In [ ]:
# add current timestamp to filename for reference
current_time = (dt.datetime.utcnow().strftime('%Y_%m_%d_%H%M%S'))

# git repo folder
git_folder = 'd:/git/example_infrastructure_data_dev'

# export folder will contain all csv exported DataFrames for Ticket Creation
export_folder = 'd:/exports/'

### Import Modules, Create env Variables

In [ ]:
# import configparser for env secrets
from configparser import ConfigParser

config = ConfigParser()
config.read(f'{git_folder}/config/env.ini')
import requests
from requests.structures import CaseInsensitiveDict

In [ ]:
# import and assign secrets from env.ini

dattormm_config = config['dattormm']

base_uri = dattormm_config['base_uri']

## Create auth token

In [ ]:
# call token api url
token_uri = f'{base_uri}/auth/oauth/token'


# construct header
headers = CaseInsensitiveDict()
headers['Content-Type'] = 'application/x-www-form-urlencoded'

# construct req body
data = CaseInsensitiveDict()
data['grant_type'] = 'password'
data['username'] = dattormm_config['api_key']
data['password'] = dattormm_config['api_secret']

# request content response
resp = requests.post(token_uri, headers=headers, data=data, auth=('public-client', 'public'))
content = resp.content.decode('utf-8')
c_dict = json.loads(content)

access_token = c_dict['access_token']

## Create DataFrame via API Call Iteration


In [ ]:

## Create Devices DataFrame
# request content response
request_url = f'{base_uri}/api/v2/account/devices'

# construct header
headers = CaseInsensitiveDict()
headers['Authorization'] = f'Bearer {access_token}'
headers['Content-Type'] = 'application/json'

# construct req body
data = ''

print(f'Request URL: {request_url}')

resp = requests.get(request_url, headers=headers, data=data)
content = resp.content.decode('utf-8')
c_dict = json.loads(content)


# iterate and combine remaining pages
df_devices = pd.DataFrame(c_dict['devices'])
while c_dict['pageDetails']['nextPageUrl']:
    next_page = c_dict['pageDetails']['nextPageUrl']
    resp = requests.get(next_page, headers=headers, data=data)
    content = resp.content.decode('utf-8')
    c_dict = json.loads(content)

    df_current_page = pd.DataFrame(c_dict['devices'])
    df_devices = pd.concat([df_devices, df_current_page], ignore_index=False)

# Data Shaping

## Create New Columns from Dictionary Columns

### Set Index to device UID

In [ ]:
df_devices.set_index('uid',inplace=True)

### Type | Category

In [ ]:
def device_category(device):
    if device == None:
        return None
    else:
        return device['category']

In [ ]:
def device_type(device):
    if device == None:
        return None
    else:
        return device['type']

In [ ]:
df_devices['category'] = df_devices['deviceType'].apply(device_category)
df_devices['type'] = df_devices['deviceType'].apply(device_type)

# Rename 'type' values to split devices into (2) : 'computer' or 'server'
#df_devices['type'].replace({'Desktop':'computer','Laptop':'computer','Server':'server'},inplace=True)

In [ ]:
df_devices.drop(columns='deviceType',inplace=True)

### Patch Managment Breakdown
 patchStatus | patchesApprovedPending | patchesNotApproved | patchesInstalled

In [ ]:
# patchStatus
def patch_status(patch_managment):
    return patch_managment['patchStatus']

df_devices['patchStatus'] = df_devices['patchManagement'].apply(patch_status)

# patchesApprovedPending
def patches_approved_pending(patch_management):
    return patch_management['patchesApprovedPending']

df_devices['patchesApprovedPending'] = df_devices['patchManagement'].apply(patches_approved_pending)

# patchesNotApproved
def patches_not_approved(patch_managment):
    return patch_managment['patchesNotApproved']

df_devices['patchesNotApproved'] = df_devices['patchManagement'].apply(patches_not_approved)

# patchesInstalled
def patches_installed(patch_management):
    return patch_management['patchesInstalled']

df_devices['patchesInstalled'] = df_devices['patchManagement'].apply(patches_installed)


# drop patchManagement {inplace=True}
df_devices.drop('patchManagement',axis=1,inplace=True)

### Patch Percent Patched Calculated Column
(Compliance Percentage = PatchesApproved / Patches Installed)

## Drop 'antivirus'

In [ ]:
df_devices.drop('antivirus',axis=1,inplace=True)

## Create Time Columns and Timedate Shaping

### Add Timezone Column from UDF

In [ ]:
# Timezone
def local_timezone(udf):
    return udf['udf10']

df_devices['localTimezone'] = df_devices['udf'].apply(local_timezone)

# drop udf {inplace=True}
df_devices.drop('udf',axis=1,inplace=True)

### Create Date Correlation Columns

In [ ]:
# all date columns
parse_dates =  ['lastAuditDate','lastSeen','lastReboot','creationDate',]

### Convert Epoch to UTC

In [ ]:
df_devices['lastAuditDate'] = pd.to_datetime(df_devices['lastAuditDate'],unit='ms',errors='coerce')
#df_devices['lastAuditDate'].head(5)

In [ ]:
df_devices['lastSeen'] = pd.to_datetime(df_devices['lastSeen'],unit='ms',errors='coerce')
#df_devices['lastSeen'].head(5)

In [ ]:
df_devices['creationDate'] = pd.to_datetime(df_devices['creationDate'],unit='ms',errors='coerce')
#df_devices['creationDate'].head(5)

In [ ]:
df_devices['lastReboot'] = pd.to_datetime(df_devices['lastReboot'],unit='ms',errors='coerce')
#df_devices['lastReboot'].head(5)

### Define and apply functions to create correlation columns

In [ ]:
def no_audit_7_days(last_audit):
    if last_audit < dt.datetime.now() - dt.timedelta(days=7):
        return 1
    else:
        return 0

In [ ]:
def no_audit_30_days(last_audit):
    if last_audit < dt.datetime.now() - dt.timedelta(days=30):
        return 1
    else:
        return 0

In [ ]:
def offline_30_days(last_seen):
    if last_seen < dt.datetime.now() - dt.timedelta(days=30):
        return 1
    else:
        return 0

In [ ]:
def no_reboot_30_days(last_reboot):
    if last_reboot < dt.datetime.now() - dt.timedelta(days=30):
        return 1
    else:
        return 0

In [ ]:
# Create Column - Devices Last Audit > 7 days
df_devices['noAudit7Days'] = df_devices['lastAuditDate'].apply(no_audit_7_days)

In [ ]:
# Create Column - Devices Last Audit > 7 days
df_devices['noAudit30Days'] = df_devices['lastAuditDate'].apply(no_audit_30_days)

In [ ]:
# Create Column - Devices Offline 30 Days
df_devices['offline30Days'] = df_devices['lastSeen'].apply(offline_30_days)

In [ ]:
# Create Column - Last Reboot Extended Duration and Online without Reboot Extended Duration
df_devices['noReboot30Days'] = df_devices['lastReboot'].apply(no_reboot_30_days)

In [ ]:
# Create DF copy for reference
df_raw_data = df_devices

In [ ]:
df_devices['patchStatusPercent'] = round( 100 - ((df_devices['patchesApprovedPending'] / ((df_devices['patchesApprovedPending'] + df_devices['patchesInstalled']) ) * 100)),2)

## Device OS EOL isEOL and daysTillEol

### EOL OS List as Parse

## DattoRMM DataFrame Data Standardization Shaping

### Hostname to_upper()

In [ ]:
df_devices['hostname'] = df_devices['hostname'].str.upper()

### Replace Dtypes with Int64

In [ ]:
convert_to_int_mask = ((df_devices.dtypes == 'float') | (df_devices.dtypes == 'bool') | (df_devices.dtypes == 'uint8')) & (df_devices.columns != 'patchStatusPercent')
convert_to_int = df_devices.dtypes[convert_to_int_mask].index.tolist()

In [ ]:
df_devices[convert_to_int] = df_devices[convert_to_int].astype('Int64')

## Add Dummy Columns

### Add 'patchStatus' Dummy Columns

In [ ]:
df_patch_status = pd.get_dummies(df_devices['patchStatus'],prefix='patchStatus',prefix_sep='')
df_patch_status.drop('patchStatusNoPolicy',axis=1, inplace=True)
df_devices = df_devices.join(df_patch_status)
df_devices.drop('patchStatus',axis=1,inplace=True)

### Add 'category' Dummy Columns

In [ ]:
df_category = pd.get_dummies(df_devices['category'],prefix='is',prefix_sep='')
df_category.drop('isDesktop',axis=1,inplace=True)
df_devices = df_devices.join(df_category)
df_devices.drop('category',axis=1,inplace=True)

## Add Report Creation Date Info Column

In [ ]:
report_creation_date = (dt.datetime.utcnow().strftime('%Y-%m-%d %H:%M:%S'))
df_devices['reportCreationDate'] = report_creation_date

# Initial Metrics and CSV Creation

## All Fields

In [ ]:
df_devices.to_csv(export_folder + 'all_fields_' + str(current_time) + '.csv',index=False)

## Reboot Issues

### Empty 'Last Reboot' Field

In [ ]:
df_devices[df_devices['lastReboot'].fillna('Missing') == 'Missing'].to_csv(export_folder + 'last_reboot_isnull_' + str(current_time) + '.csv',index=False)

### No Reboot > 60 Days

In [ ]:
df_devices[df_devices['noReboot30Days'] == 1].to_csv(export_folder + 'no_reboot_30_days_' + str(current_time) + '.csv',index=False)

### No Reboot Since Last Year

In [ ]:
df_devices[df_devices['lastReboot'].dt.year == dt.datetime.utcnow().year - 1].to_csv(export_folder + 'no_reboot_current_yr_' + str(current_time) + '.csv',index=False)

### No Reboot in 2 Years

In [ ]:
df_devices[df_devices['lastReboot'].dt.year == dt.datetime.utcnow().year - 2].to_csv(export_folder + 'no_reboot_2yrs_' + str(current_time) + '.csv',index=False)

## DattoRMM Audit Issues

### Empty 'Last Audit' Field

In [ ]:
df_devices[df_devices['lastAuditDate'].fillna('Missing') == 'Missing'].to_csv(export_folder + 'last_audit_isnull_' + str(current_time) + '.csv',index=False)

### No Audit > 7 days

In [ ]:
df_devices[df_devices['noAudit7Days'] == 1].to_csv(export_folder + 'no_audit_7days_' + str(current_time) + '.csv',index=False)

### No Audit > 30 days

In [ ]:
df_devices[df_devices['noAudit30Days'] == 1].to_csv(export_folder + 'no_audit_30days_' + str(current_time) + '.csv',index=False)

### No Audit Since Last Year

In [ ]:
df_devices[df_devices['lastAuditDate'].dt.year == dt.datetime.utcnow().year - 1].to_csv(export_folder + 'no_audit_current_yr_' + str(current_time) + '.csv',index=False)

## Patch Status

### Install Error Status and Online > 30 days

In [ ]:
df_devices[(df_devices['patchStatusInstallError'] == 1) & (df_devices['offline30Days'] == 0)].to_csv(export_folder + 'patchStatus_InstallError_isonline_' + str(current_time) + '.csv',index=False)

### Reboot Required Status and Online > 30 days

In [ ]:
df_devices[(df_devices['patchStatusRebootRequired'] == 1) & (df_devices['offline30Days'] == 0)].to_csv(export_folder + 'patchStatus_RebootRequired_isonline_' + str(current_time) + '.csv',index=False)

### No Data Status and Online > 30 days

In [ ]:
df_devices[(df_devices['patchStatusNoData'] == 1) & (df_devices['offline30Days'] == 0)].to_csv(export_folder + 'patchStatus_NoData_isonline_' + str(current_time) + '.csv',index=False)

## Patch Percentage

### Patch Status Percentage under 95%

In [ ]:
df_devices[df_devices['patchStatusPercent'] < 95].to_csv(export_folder + 'patchStatus_Under_95_' + str(current_time) + '.csv',index=False)

# Service Delivery Reports
(ticket creation report)

## MS Patching Service Report Shaping Criteria

In [ ]:
df = pd.read_csv(f'{git_folder}/dictionaries/datto_rmm_ms_patching_report.dict')
ms_patching = {}
ms_patching['siteNames'] = list(df['siteName'].dropna())
df = pd.read_csv(f'{git_folder}/dictionaries/service_delivery_standard_columns.dict')
ms_patching['columns'] = list(df['columns'].dropna())

## Create Filtered Service Delivery DataFrame

In [ ]:
df_service_delivery = df_devices[df_devices['siteName'].isin(ms_patching['siteNames'])]
df_service_delivery = df_service_delivery[ms_patching['columns']]

### Service Delivery - No Audit or No Data > 30 Days (not including offline30Days)

In [ ]:
df_service_delivery[((df_service_delivery['noAudit30Days'] == 1) | (df_service_delivery['patchStatusNoData'] == 1)) & df_service_delivery['offline30Days'] == 0].to_csv(export_folder + 'service_delivery_no_audit_no_data_30days_' + str(current_time) + '.csv')

### Service Delivery - Offline > 30 Days

In [ ]:
df_service_delivery[df_service_delivery['offline30Days'] == 1].to_csv(export_folder + 'service_delivery_offline_30days_' + str(current_time) + '.csv')

### Service Delivery - Patch Status Percentage under 95% and Online > 30 days

In [ ]:
df_service_delivery[(df_service_delivery['patchStatusPercent'] < 95) & (df_service_delivery['offline30Days'] == 0)].to_csv(export_folder + 'service_delivery_patchStatus_Under_95_isOnline_' + str(current_time) + '.csv')

### Service Delivery - Device EOL (out of compliance)

In [ ]:
df_service_delivery[df_service_delivery['isEOL'] == 1].to_csv(export_folder + 'service_delivery_eol_devices_' + str(current_time) + '.csv')